In [ ]:
!pip install ultralytics

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# 🔁 Define paths (update if needed)
train_ann_path = "/kaggle/input/weapon-detection-dataset-02/train/VOC2007/Annotations"
valid_ann_path = "/kaggle/input/weapon-detection-dataset-02/valid/VOC2007/Annotations"

train_lbl_path = "/kaggle/working/train_labels"
valid_lbl_path = "/kaggle/working/valid_labels"

os.makedirs(train_lbl_path, exist_ok=True)
os.makedirs(valid_lbl_path, exist_ok=True)

# 🔖 Class names (must match the names in your dataset exactly!)
classes = ['grenade', 'knife', 'pistol', 'rifle']  # 0-indexed

# 🔁 Conversion function
def convert_voc_to_yolo(ann_dir, output_dir):
    for xml_file in tqdm(os.listdir(ann_dir)):
        if not xml_file.endswith(".xml"):
            continue

        tree = ET.parse(os.path.join(ann_dir, xml_file))
        root = tree.getroot()

        size = root.find("size")
        w = int(size.find("width").text)
        h = int(size.find("height").text)

        lines = []

        for obj in root.findall("object"):
            cls = obj.find("name").text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)

            xml_box = obj.find("bndbox")
            xmin = int(xml_box.find("xmin").text)
            ymin = int(xml_box.find("ymin").text)
            xmax = int(xml_box.find("xmax").text)
            ymax = int(xml_box.find("ymax").text)

            # YOLO format: class x_center y_center width height (normalized)
            x_center = (xmin + xmax) / 2 / w
            y_center = (ymin + ymax) / 2 / h
            box_w = (xmax - xmin) / w
            box_h = (ymax - ymin) / h

            line = f"{cls_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}"
            lines.append(line)

        # Save as .txt
        txt_filename = os.path.splitext(xml_file)[0] + ".txt"
        with open(os.path.join(output_dir, txt_filename), "w") as f:
            f.write("\n".join(lines))

# 🔄 Run conversion for train and valid sets
convert_voc_to_yolo(train_ann_path, train_lbl_path)
convert_voc_to_yolo(valid_ann_path, valid_lbl_path)

print("Conversion complete!")


In [ ]:
import os
import shutil

# Define paths
train_img_src = "/kaggle/input/weapon-detection-dataset-02/train/VOC2007/JPEGImages"
valid_img_src = "/kaggle/input/weapon-detection-dataset-02/valid/VOC2007/JPEGImages"

train_lbl_src = "/kaggle/working/train_labels"
valid_lbl_src = "/kaggle/working/valid_labels"

train_dst = "/kaggle/working/train"
valid_dst = "/kaggle/working/valid"

os.makedirs(train_dst + "/images", exist_ok=True)
os.makedirs(train_dst + "/labels", exist_ok=True)
os.makedirs(valid_dst + "/images", exist_ok=True)
os.makedirs(valid_dst + "/labels", exist_ok=True)

# Move images
for f in os.listdir(train_img_src):
    if f.endswith(".jpg") or f.endswith(".png"):
        shutil.copy(os.path.join(train_img_src, f), train_dst + "/images")

for f in os.listdir(valid_img_src):
    if f.endswith(".jpg") or f.endswith(".png"):
        shutil.copy(os.path.join(valid_img_src, f), valid_dst + "/images")

# Move labels
for f in os.listdir(train_lbl_src):
    shutil.copy(os.path.join(train_lbl_src, f), train_dst + "/labels")

for f in os.listdir(valid_lbl_src):
    shutil.copy(os.path.join(valid_lbl_src, f), valid_dst + "/labels")

print("Images and labels are organized for YOLOv8.")


In [ ]:
import os

image_dir = '/kaggle/working/valid/images'
label_dir = '/kaggle/working/valid/labels'

img_files = [f.replace('.jpg', '.txt') for f in os.listdir(image_dir) if f.endswith('.jpg')]
lbl_files = os.listdir(label_dir)

missing_labels = [f for f in img_files if f not in lbl_files]

print(f"Total images: {len(img_files)}")
print(f"Matching labels: {len(lbl_files)}")
print(f"Missing labels: {len(missing_labels)}")


In [ ]:
import shutil

src = "/kaggle/working/valid_labels"
dst = "/kaggle/working/valid/labels"
os.makedirs(dst, exist_ok=True)

for f in os.listdir(src):
    shutil.copy(os.path.join(src, f), dst)


In [ ]:
yaml_content = """
train: /kaggle/working/train/images
val: /kaggle/working/valid/images

nc: 4
names: ['Handgun', 'Knife', 'Short_rifle', 'Rifle']
"""

with open("/kaggle/working/data.yaml", "w") as f:
    f.write(yaml_content.strip())

print("Updated data.yaml saved.")


In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# 💡 Class names exactly as found in the XML
classes = ['Handgun', 'Knife', 'Short_rifle', 'Rifle']

# 💾 Paths
ann_dir = "/kaggle/input/weapon-detection-dataset-02/valid/VOC2007/Annotations"
label_dir = "/kaggle/working/valid_labels"
os.makedirs(label_dir, exist_ok=True)

def convert_voc_to_yolo(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)

    lines = []

    for obj in root.findall("object"):
        cls = obj.find("name").text.strip()
        if cls not in classes:
            print(f"⚠️ Skipped unknown class: {cls}")
            continue

        cls_id = classes.index(cls)
        xml_box = obj.find("bndbox")

        xmin = int(float(xml_box.find("xmin").text))
        ymin = int(float(xml_box.find("ymin").text))
        xmax = int(float(xml_box.find("xmax").text))
        ymax = int(float(xml_box.find("ymax").text))

        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        box_width = (xmax - xmin) / w
        box_height = (ymax - ymin) / h

        line = f"{cls_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"
        lines.append(line)
        print(f"Added: {line}")

    return lines

# 🔁 Process all XML files
for xml_file in tqdm(os.listdir(ann_dir)):
    if not xml_file.endswith(".xml"):
        continue

    xml_path = os.path.join(ann_dir, xml_file)
    yolo_lines = convert_voc_to_yolo(xml_path)

    txt_file = os.path.splitext(xml_file)[0] + ".txt"
    with open(os.path.join(label_dir, txt_file), "w") as f:
        f.write("\n".join(yolo_lines))


In [ ]:
!cat /kaggle/working/valid_labels/0045a7e07dd4fa0e.txt


In [ ]:
import shutil

src = "/kaggle/working/valid_labels"
dst = "/kaggle/working/valid/labels"
os.makedirs(dst, exist_ok=True)

for f in os.listdir(src):
    shutil.copy(os.path.join(src, f), dst)

print("All validation labels copied successfully.")


In [ ]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

classes = ['Handgun', 'Knife', 'Short_rifle', 'Rifle']

ann_dir = "/kaggle/input/weapon-detection-dataset-02/train/VOC2007/Annotations"
label_dir = "/kaggle/working/train_labels"
os.makedirs(label_dir, exist_ok=True)

def convert_voc_to_yolo(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)

    lines = []

    for obj in root.findall("object"):
        cls = obj.find("name").text.strip()
        if cls not in classes:
            continue  # Skip unmatched classes

        cls_id = classes.index(cls)
        bbox = obj.find("bndbox")

        xmin = int(float(bbox.find("xmin").text))
        ymin = int(float(bbox.find("ymin").text))
        xmax = int(float(bbox.find("xmax").text))
        ymax = int(float(bbox.find("ymax").text))

        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        box_width = (xmax - xmin) / w
        box_height = (ymax - ymin) / h

        line = f"{cls_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}"
        lines.append(line)

    return lines

# 🔁 Process all training XMLs
for xml_file in tqdm(os.listdir(ann_dir)):
    if not xml_file.endswith(".xml"):
        continue

    xml_path = os.path.join(ann_dir, xml_file)
    yolo_lines = convert_voc_to_yolo(xml_path)

    txt_file = os.path.splitext(xml_file)[0] + ".txt"
    with open(os.path.join(label_dir, txt_file), "w") as f:
        f.write("\n".join(yolo_lines))


In [ ]:
import os

label_dir = "/kaggle/working/train_labels"
non_empty = [f for f in os.listdir(label_dir) if os.path.getsize(os.path.join(label_dir, f)) > 0]

print(f"Total label files: {len(os.listdir(label_dir))}")
print(f"Non-empty label files: {len(non_empty)}")


In [ ]:
import shutil

src = "/kaggle/working/train_labels"
dst = "/kaggle/working/train/labels"
os.makedirs(dst, exist_ok=True)

for f in os.listdir(src):
    shutil.copy(os.path.join(src, f), dst)

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=/kaggle/working/data.yaml epochs=30 imgsz=640

In [ ]:
!yolo task=detect mode=val model=runs/detect/train/weights/best.pt data=/kaggle/working/data.yaml

In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("runs/detect/train/weights/best.pt")  # Make sure this file exists


In [ ]:
results = model("/kaggle/working/valid/images/0045a7e07dd4fa0e.jpg")
results[0].show()


In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
import os

model = YOLO("runs/detect/train/weights/best.pt")

image_dir = "/kaggle/working/valid/images"
output_dir = "/kaggle/working/predictions"
os.makedirs(output_dir, exist_ok=True)

image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".jpg")]

# Inference one by one
for i, path in enumerate(image_paths[:10]):  # ← reduce if needed
    result = model(path)[0]  # get single result object
    save_path = os.path.join(output_dir, f"pred_{i}.jpg")
    result.save(filename=save_path)
    display(Image(filename=save_path))

print("Individual image predictions completed.")


In [ ]:
import requests
from PIL import Image
from io import BytesIO
from ultralytics import YOLO
from IPython.display import Image as IPyImage, display  # renamed to avoid conflict with PIL.Image
import os

# Step 1: Download image from the web
url = "https://www.outdoorlife.com/wp-content/uploads/2021/07/13/Glock-17m.jpeg?strip=all&quality=85"
response = requests.get(url)

# Step 2: Open and save image locally
img = Image.open(BytesIO(response.content)).convert("RGB")
img_path = "/kaggle/working/web_handgun.jpg"
img.save(img_path)

# Step 3: Load your trained YOLOv8 model
model = YOLO("runs/detect/train/weights/best.pt")  # adjust path if needed

# Step 4: Run inference on the image
result = model(img_path, conf=0.25)[0]  # conf threshold can be adjusted

# Step 5: Save prediction result image
output_path = "/kaggle/working/pred_web_handgun.jpg"
result.save(filename=output_path)

# Step 6: Display the predicted image
display(IPyImage(filename=output_path))


In [ ]:
from shutil import copyfile
copyfile("runs/detect/train/weights/best.pt", "/kaggle/working/yolov8-weapon-detector.pt")

In [ ]:
from shutil import copyfile
copyfile("runs/detect/train/weights/best.pt", "/kaggle/working/yolov8-weapon-detector.pt")


In [ ]:
shutil.make_archive("/kaggle/working/yolo_labels", 'zip', "/kaggle/working/train/labels")

In [ ]:
!pip install scikit-learn

In [ ]:
# Step 1: Import required metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Step 2: Define your ground truth and predicted labels
# Replace these with your actual labels
y_true = [0, 1, 1, 2, 0, 1]  # ground truth class indices
y_pred = [0, 0, 1, 2, 0, 1]  # predicted class indices by YOLO

# Step 3: Compute metrics
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
accuracy = accuracy_score(y_true, y_pred)

# Step 4: Print results
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

In [ ]:
from ultralytics import YOLO
import os
import random
from PIL import Image
from IPython.display import display

# Load your trained YOLOv8 model
model = YOLO("runs/detect/train/weights/best.pt")

# Directory where your validation images are stored
image_dir = "/kaggle/working/valid/images"

# List all image filenames
image_files = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".jpeg", ".png"))]

# Ensure we don't exceed the number of available images
sample_count = min(25, len(image_files))

# Select 25 random images
random_images = random.sample(image_files, sample_count)

# Directory to save predictions
output_dir = "/kaggle/working/predictions_batch"
os.makedirs(output_dir, exist_ok=True)

# Loop through each selected image, predict, save, and display
for i, img_name in enumerate(random_images):
    image_path = os.path.join(image_dir, img_name)
    result = model(image_path)[0]
    output_path = os.path.join(output_dir, f"pred_{i+1}.jpg")
    result.save(filename=output_path)
    display(Image.open(output_path))